In [1]:
class VerySafe:
    
    def _get_attr(self):
        return self._x
    
    def _set_attr(self, x):
        assert x > 0
        self._x = x
        
    def _del_attr(self):
        del self._x
        
    x = property(_get_attr, _set_attr, _del_attr)

In [2]:
very_safe = VerySafe()

In [3]:
very_safe.x = 42

In [4]:
very_safe.x

42

In [5]:
very_safe.x = -42

AssertionError: 

In [6]:
very_safe.y = -42

In [7]:
very_safe.y

-42

### Descriptor `NonNegative`:

In [8]:
class NonNegative:
    def __get__(self, instance, owner):
        return magically_get_value(...)
    
    def __set__(self, instance, owner):
        assert value >= 0, "non-negative value required"
        magically_set_value(...)
        
    def __delete__(self, instance):
        magically_delete_value(...)

In [9]:
class VerySafe:
    x = NonNegative()
    y = NonNegative()

In [10]:
class Descr:
    def __get__(self, instance, owner):
        print(instance, owner)

In [11]:
class A:
    attr = Descr()

In [12]:
A.attr  # instance=None, owner=A

None <class '__main__.A'>


In [13]:
A().attr  # instance=A(), owner=A

<__main__.A object at 0x7f033826f710> <class '__main__.A'>


In [14]:
class B:
    attr = Descr()

In [15]:
B.attr

None <class '__main__.B'>


In [16]:
B().attr

<__main__.B object at 0x7f0338274810> <class '__main__.B'>


In [17]:
class Descr:
    def __set__(self, instance, value):
        print(instance, value)

In [18]:
class A:
    attr = Descr()

In [19]:
instance = A()

In [20]:
instance.attr = 42

<__main__.A object at 0x7f03381f8950> 42


In [21]:
A.attr = 42

In [22]:
class Descr:
    def __delete__(self, instance):
        print(instance)
        
class A:
    attr = Descr()

del A().x

AttributeError: x

In [23]:
del A.x

AttributeError: x

### Example: data descriptor

In [24]:
class Descr:
    def __get__(self, instance, owner):
        print("Descr.__get__")
    
    def __set__(self, instance, value):
        print("Descr.__set__")

In [25]:
class A:
    attr = Descr()

In [26]:
instance = A()

In [27]:
instance.attr

Descr.__get__


In [28]:
instance.__dict__["attr"] = 42

In [29]:
instance.attr

Descr.__get__


### Example: non-data descriptor

In [30]:
class Descr:
    def __get__(self, instance, owner):
        print("Descr.__get__")

In [31]:
class A:
    attr = Descr()

In [32]:
instance = A()

In [33]:
instance.attr

Descr.__get__


In [34]:
instance.__dict__["attr"] = 42

In [35]:
instance.attr

42

In [36]:
class CachedProperty:
    
    def __init__(self, method):
        self._method = method
        
    def __get__(self, instance, owner):
        if instance is None:
            return self
        value = self._method(instance)
        setattr(instance, self._method.__name__, value)
        return value

In [37]:
class A:
    @CachedProperty
    def f(self):
        print("computing")
        return 42

In [38]:
A().f

computing


42

In [39]:
a = A()

In [40]:
a.f

computing


42

In [41]:
a.f

42

Where to keep data?
The data may be kept in attributes of descriptor itself:

In [42]:
class Proxy:
    
    def __get__(self, instance, owner):
        return self.value
    
    def __set__(self, instance, value):
        self.value = value
        
    def __delete__(self, instance):
        del self.value

In [43]:
class Something:
    attr = Proxy()

In [44]:
some = Something()

In [45]:
some.attr = 42

In [46]:
other = Something()

In [47]:
other.attr

42

Now in dictionary (instance is hashable):

In [48]:
class Proxy:
    
    def __init__(self):
        self.data = {}
        
    def __get__(self, instance, owner):
        if instance is None:
            return self
        if instance not in self.data:
            raise AttributeError
        return self.data[instance]
    
    def __set__(self, instance, value):
        self.data[instance] = value
        
    def __delete__(self, instance):
        del self.data[instance]

Now keeping in instance:

In [49]:
class Proxy:
     
    def __init__(self, label):
        self.label = label
        
    def __get__(self, instance, owner):
        if instance is None:
            return self
        return instance.__dict__[self.label]
    
    def __set__(self, instance, value):
        instance.__dict__[self.label] = value
        
    def __delete__(self, instance):
        del instance.__dict__[self.label]

In [50]:
class Something:
    attr = Proxy("attr")

In [51]:
some = Something()

In [52]:
some.attr = 42

In [53]:
some.attr

42

### Examples of descriptors:

In [54]:
# class property:
#     def __init__(self, get=None, set=None, delete=None):
#         self._get = get
#         self._set = set
#         self._delete = delete
        
#     def __get__(self, instance, owner):
#         if self._get is None:
#             raise AttributeError("undreadable attribute")
#         return self._get(instance)
    
# class Something:
#     @property
#     def attr(self):
#         return 42

In [55]:
class Something:
    
    def do_something(self):
        pass

In [56]:
Something.do_something

<function __main__.Something.do_something(self)>

In [57]:
Something().do_something

<bound method Something.do_something of <__main__.Something object at 0x7f032a3f3810>>

In [58]:
from types import MethodType

In [59]:
class Function:
    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return MethodType(self, instance, owner)

Functions are descriptors!

Decorator `staticmethod`:

In [60]:
class SomeClass:
    @staticmethod
    def do_something():
        pass

In [61]:
SomeClass.do_something()

In [ ]:
# class staticmethod:
#     def __init__(self, method):
#         self._method = method
        
#     def __get__(self, instance, owner):
#         return self._method
    
# class Something:
#     @staticmethod
#     def do_something():
#         print("I'm busy, alright?")

Class method `classmethod`. This may be useful for alternative constructors.

In [63]:
class Settings:
    @classmethod
    def read_from(cls, path):
        return cls()

In [64]:
Settings.read_from

<bound method Settings.read_from of <class '__main__.Settings'>>

In [ ]:
# import functools

# class classmethod:
#     def __init__(self, method):
#         self._method = method
        
#     def __get__(self, instance, owner):
#         return functools.partial(self._method, owner)
    
# class Something:
#     @classmethod
#     def do_something(cls):
#         print("Called with ", cls)

### Metaclasses

In [65]:
class Something:
    attr = 42

In [66]:
Something

__main__.Something

In [67]:
type(Something)

type

In [68]:
name, bases, attrs = "Something", (), {"attr": 42}

In [69]:
Something = type(name, bases, attrs)

In [70]:
Something

__main__.Something

In [71]:
some = Something()

In [72]:
some.attr

42

In [73]:
class Meta(type):
    def some_method(cls):
        return "foobar"

In [74]:
class Something(metaclass=Meta):
    attr = 42

In [75]:
type(Something)

__main__.Meta

In [76]:
Something.some_method

<bound method Meta.some_method of <class '__main__.Something'>>

In [77]:
Something().some_method

AttributeError: 'Something' object has no attribute 'some_method'

In [78]:
class Something(Base, metaclass=Meta):
    def __init__(self, attr):
        self.attr = attr
        
    def do_something(self):
        pass


NameError: name 'Base' is not defined

`__new__`:

In [79]:
class Noop:
    
    def __new__(cls, *args, **kwargs):
        print("Creating instance with {} and {}".format(args, kwargs))
        instance = super().__new__(cls)
        return instance
    
    def __init__(self, *args, **kwargs):
        print("Initializing with {} and {}".format(args, kwargs))

In [80]:
noop = Noop(42, attr="value")

Creating instance with (42,) and {'attr': 'value'}
Initializing with (42,) and {'attr': 'value'}


`__new__` creates the instance and returns it, while `__init__` initializes it.

Example of useless metaclass:

In [81]:
from collections import OrderedDict

In [82]:
class UselessMeta(type):
    def __new__(metacls, name, bases, clsdict):
        print(type(clsdict))
        print(list(clsdict))
        cls = super().__new__(metacls, name, bases, clsdict)
        return cls
    
    @classmethod
    def __prepare__(metacls, name, bases):
        return OrderedDict()

In [83]:
class Something(metaclass=UselessMeta):
    attr = "foo"
    other_attr = "bar"

<class 'collections.OrderedDict'>
['__module__', '__qualname__', 'attr', 'other_attr']


In [84]:
import abc

In [85]:
class Iterable(metaclass=abc.ABCMeta):
    @abc.abstractmethod
    def __iter__(self):
        pass
    
class Something(Iterable):
    pass

In [86]:
Something()

TypeError: Can't instantiate abstract class Something with abstract methods __iter__

In [88]:
from collections import deque

class MemorizingDict(dict):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._history = deque(maxlen=10)
        
    def __setitem__(self, key, value):
        self._history.append(key)
        super().__setitem__(key, value)
    
    def get_history(self):
        return self._history

In [89]:
d = MemorizingDict({"foo": 42})
d.setdefault("bar", 24)

24

In [90]:
d["baz"] = 100500

In [91]:
print(d.get_history())

deque(['baz'], maxlen=10)


In [92]:
from collections import abc

In [93]:
issubclass(list, abc.Sequence)

True

In [94]:
isinstance({}, abc.Hashable)

False

In [95]:
def flatten(obj):
    for item in obj:
        if isinstance(item, abc.Iterable):
            yield from flatten(item)
        else:
            yield item

In [96]:
list(flatten([[1, 2], 3, [], [4]]))

[1, 2, 3, 4]